In [7]:
#dependencies 
#had to 
import pandas as pd 
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import config 
from coincallandwritepythonfile import refined_coin_list

In [9]:
#get ids for each coin and convert to list
coin_ids = refined_coin_list.id.to_list()
type(coin_ids)
#getting 5 first ids to try in practice run
five_ids_to_try = coin_ids[:5]

In [137]:
def get_coin_info(list_of_coins_, api_key):
    """Function makes API call for each coin ID within list of coins.
    Designed to take list from top 100 coins and get their most recent price information. """

    #create an empty dataframe that we will append responses too
    df = pd.DataFrame()
    #a count variable to index the respones when trying to concat 
    count =0
    for id in list_of_coins_:
        url = 'https://pro-api.coinmarketcap.com/v2/cryptocurrency/quotes/latest'
        parameters = {
            'id': id
            }
        headers = {
        'Accepts': 'application/json',
        'X-CMC_PRO_API_KEY': api_key
        }

        #make API call and return response as a dictionary
        session = Session()
        session.headers.update(headers)
        response = session.get(url, params=parameters)
        data = json.loads(response.text)
        
        #parse dict and load in relavant data to a DF
        cmc_df = pd.DataFrame.from_dict(data['data'], orient='index')
         
        #expand nested quote column containing price information
        expanded_df = pd.DataFrame.from_dict(cmc_df['quote'][0], orient= 'index')
        

        
        #set then index for count ie the number of runs 
        expanded_df.set_index(pd.Index([count]),inplace= True)

        #dropping nested columns that we won't explore during this exercise
        trial = cmc_df.drop(['quote','tags', 'platform'], axis= 1)
          
        #setting index as count variable so can join back
        trial.set_index(pd.Index([count]),inplace= True)
        
        #concat was tricky had to add
        #had to add key to get them to append properly
        coin_price_df = pd.concat([trial,expanded_df], axis=1, keys=['trial', 'expanded_df']  )
        #coin_price_df.reset_index(drop = True, inplace= True)
        coin_price_merge = pd.merge(trial, expanded_df)
        #this saves it as a coreframe

        #going to try to append this to another dataframe
        #df = pd.DataFrame()
        #coin_price_df.set_index(pd.Index([count]),inplace= True)
        
        df = df.append(coin_price_merge)

        #df = df.append(coin_price_df)


        count +=1
        #df.set_index(pd.Index([count]),inplace= True)
    #had to drop level inorder to get ride of multilevel index   
    # #https://sparkbyexamples.com/pandas/pandas-multiindex-dataframe-examples/?msclkid=fba45bb6c10e11eca1d376f5b274bab7 
    
    
    #df = df.droplevel(0, axis =1)
    return df

        


In [138]:
df = get_coin_info(five_ids_to_try,  config.coin_api_key)

In [139]:
df

,id,name,symbol,slug,num_market_pairs,date_added,max_supply,circulating_supply,total_supply,is_active,...,volume_change_24h,percent_change_1h,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap
0,1,Bitcoin,BTC,bitcoin,9357,2013-04-28T00:00:00.000Z,21000000,1.901909e+07,1.901909e+07,1,...,-9.2906,0.051088,-2.799836,-2.051057,-6.586973,5.289726,11.891771,7.533151e+11,40.7291,8.317755e+11
0,1027,Ethereum,ETH,ethereum,5659,2015-08-07T00:00:00.000Z,None,1.204864e+08,1.204864e+08,1,...,-8.2355,-0.099676,-1.394107,-2.203590,-1.058806,13.253093,21.616547,3.573333e+11,19.3177,3.573333e+11
0,825,Tether,USDT,tether,32227,2015-02-25T00:00:00.000Z,None,8.306920e+10,8.561395e+10,1,...,-11.5642,0.007491,0.002829,-0.001643,-0.016244,-0.027717,0.005706,8.308867e+10,4.4923,8.563401e+10
0,1839,BNB,BNB,bnb,813,2017-07-25T00:00:00.000Z,165116760,1.632770e+08,1.632770e+08,1,...,-2.8176,-0.227735,0.239985,-2.214106,0.567210,11.129498,12.376621,6.660225e+10,3.6009,6.735271e+10
0,3408,USD Coin,USDC,usd-coin,3790,2018-10-08T00:00:00.000Z,None,4.989235e+10,4.989235e+10,1,...,-4.1392,0.003145,0.015981,0.031693,-0.006726,0.048140,-0.021855,4.989603e+10,2.6977,4.989603e+10
